# ECS713: Week 11 

# Monads - The Log Monad

## Learning Objectives

By the end of this notebook you should: 
- understand the construction of the Log monad
- have seen examples of its use

## The basic construction

The Log monad allows the logging of computations. This is a simple example. 

As is normal with monads, we begin by defining the basic type constructor as a datatype. In this instance we have a simple form of the Log monad in which the Log is just structured as a String. There are other possibilities. It could be structured, for example, as a list of entries, where the entries themselves could either be Strings or possibly instances of a more specialised datatype. 

In [2]:
data Log a = Log String a deriving (Eq,Show)

The next two functions are defined for any monad. They are in the standard base (in this case in Control.Monad), but are not in the Prelude. They implement sequential application and the maps function. See http://hackage.haskell.org/package/base-4.14.0.0/docs/Control-Monad.html for documentation: 

In [3]:
ap mf mx = do {x <- mx; f <- mf; return (f x)}
liftM f mx = do {x <- mx; return (f x)}

Line 2: Use <$>
Found:
do x <- mx
   return (f x)
Why not:
do f <$> mx

Now we need to turn the Log datatype into an instance of Monad, and of course also Applicative and Functor with standard boilerplate to do that. 

In [4]:
-- make Monad out of Log
instance Monad Log where
-- compose Logs by concatenating entries, with separator "; ""
  (Log currentLog a) >>= f =
    let Log newEntry b = f a in Log (currentLog++newEntry) b
-- return has empty string for Log 
  return a = Log "" a

-- standard boilerplate to get Applicative 
instance Applicative Log where
  pure = return 
  (<*>) = ap

-- standard boilerplate to get Functor 
instance Functor Log where
  fmap = liftM

Line 7: Eta reduce
Found:
return a = Log "" a
Why not:
return = Log ""

## First use of the Log Monad

We can use the log monad to log a sequence of actions:

In [5]:
do
  x0 <- Log ("initial value = "++(show 0)++"; ") 0
  x1 <- Log ("add "++(show 3)++"; ") (x0+3)
  Log ("multiply by "++(show 2)++"; ") (x1*2)

Line 2: Redundant bracket
Found:
(show 0) ++ "; "
Why not:
show 0 ++ "; "Line 3: Redundant bracket
Found:
(show 3) ++ "; "
Why not:
show 3 ++ "; "Line 4: Redundant bracket
Found:
(show 2) ++ "; "
Why not:
show 2 ++ "; "

Log "initial value = 0; add 3; multiply by 2; " 6

The actions here are `start with 0`, `add 3` and `multiply by 2`, and can be seen on the right. Note there is no `return` statement, but we could give add that (redundantly): 

In [6]:
do
  x0 <- Log ("initial value = "++(show 0)++"; ") 0
  x1 <- Log ("add "++(show 3)++"; ") (x0+3)
  x2 <- Log ("multiply by "++(show 2)++"; ") (x1*2)
  return x2

Line 1: Redundant return
Found:
do x0 <- Log ("initial value = " ++ (show 0) ++ "; ") 0
   x1 <- Log ("add " ++ (show 3) ++ "; ") (x0 + 3)
   x2 <- Log ("multiply by " ++ (show 2) ++ "; ") (x1 * 2)
   return x2
Why not:
do x0 <- Log ("initial value = " ++ (show 0) ++ "; ") 0
   x1 <- Log ("add " ++ (show 3) ++ "; ") (x0 + 3)
   Log ("multiply by " ++ (show 2) ++ "; ") (x1 * 2)Line 2: Redundant bracket
Found:
(show 0) ++ "; "
Why not:
show 0 ++ "; "Line 3: Redundant bracket
Found:
(show 3) ++ "; "
Why not:
show 3 ++ "; "Line 4: Redundant bracket
Found:
(show 2) ++ "; "
Why not:
show 2 ++ "; "

Log "initial value = 0; add 3; multiply by 2; " 6

To make things clearer, let's wrap these actions up in some functions: 

In [7]:
-- three functions to log inititalisation, addition and multiplication
logInit :: Show a => a -> Log a
logInit n = Log ("initial value = "++ show n ++"; ") n

logAdd :: (Show a, Num a) => a -> a -> Log a 
logAdd m n = Log ("add "++ show m ++"; ") (n+m)

logMult :: (Show a, Num a) => a -> a -> Log a 
logMult m n = Log ("multiply by "++ show m ++ "; ") (n*m)

So our code becomes: 

In [8]:
do
  x <- logInit 0
  y <- logAdd 3 x
  logMult 2 y

Log "initial value = 0; add 3; multiply by 2; " 6

And if we put it into the bind pipeline style: 

In [9]:
logInit 0 >>= logAdd 3 >>= logMult 2

Log "initial value = 0; add 3; multiply by 2; " 6

**Exercise 1.** Write code to log the following sequence of actions: 
- start with 3
- add 6 
- multiply by 8
- subtract 2

## Another use: `foldr` and `foldl`

We can use the Log monad to see how `foldr` and `foldl` iterate over the elements of a list. 

Here is the code for `foldr`:

```-- if the list is empty, the result is the initial value z; else 
-- apply f to the first element and the result of folding the rest 
foldr f z [] = z
foldr f z (x:xs) = f x (foldr f z xs)```

We refactor this in more monadic style: 

In [10]:
foldr' f z [] = z
foldr' f z (x:xs) = 
  do
    a <- foldr' f z xs
    f x a   

In [11]:
:t foldr'

foldr' :: forall (m :: * -> *) t1 t2. Monad m => (t1 -> t2 -> m t2) -> m t2 -> [t1] -> m t2

In [12]:
foldr' logAdd (logInit 10) [1..4]

Log "initial value = 10; add 4; add 3; add 2; add 1; " 20

Or if we want to track the calls to the accumulator function, we redefine the Log produced by `logAdd`: 

In [13]:
logPlus x y = Log (show x ++ "+" ++ show y ++ "; ") (x+y)

In [14]:
foldr' logPlus (logInit 10) [1..4]

Log "initial value = 10; 4+10; 3+14; 2+17; 1+19; " 20

As we might expect, the types don't work with `foldr` iteself: 

In [15]:
foldr logPlus (logInit 10) [1..4]

: 

We can handle `foldl` similarly. 

Here is the code for `foldl`:

```-- if the list is empty, the result is the initial value; else
-- we recurse immediately, making the new initial value the result 
-- of combining the old initial value with the first element. 
foldlf z [] = z
foldl f z (x:xs) = foldl f (f z x) xs```

The translation into monadic style is: 

In [16]:
foldl' f z [] = z
foldl' f z (x:xs) = 
  do
    z' <- z   
    let a = f z' x
    foldl' f a xs

In [17]:
foldr' logPlus (logInit 10) [1..4]
foldl' logPlus (logInit 10) [1..4]

Log "initial value = 10; 4+10; 3+14; 2+17; 1+19; " 20

Log "initial value = 10; 10+1; 11+2; 13+3; 16+4; " 20

In [18]:
:t foldl'

foldl' :: forall (m :: * -> *) a1 a2. Monad m => (a1 -> a2 -> m a1) -> m a1 -> [a2] -> m a1

In [19]:
:t foldr'

foldr' :: forall (m :: * -> *) t1 t2. Monad m => (t1 -> t2 -> m t2) -> m t2 -> [t1] -> m t2

So we can see that foldl and foldt work through the list in opposite directions. 

## Exercises

**2.** A classic fold example is the use of fold to define `concat` by iterating append, either `foldl` or `foldr` works (because append is associative and `[]` is its unit):

In [20]:
concat' = foldr (++) []
concat'' = foldl (++) []

Line 1: Use concat
Found:
foldr (++) []
Why not:
concatLine 2: Use concat
Found:
foldl (++) []
Why not:
concat

In [21]:
concat' ["cat","dog","gnu"]
concat'' ["cat","dog","gnu"]

"catdoggnu"

"catdoggnu"

Use the Log monad to log these two, using the functions `foldr'` and `foldl'`. You will need to implement a logged version of append (`++`). Here are sample logs: 

```Log "initial value = \"\"; \"\"++\"cat\"; \"cat\"++\"dog\"; \"catdog\"++\"gnu\"; " "catdoggnu"```

```Log "initial value = \"\"; \"gnu\"++\"\"; \"dog\"++\"gnu\"; \"cat\"++\"doggnu\"; " "catdoggnu"```


In [35]:
logApp x y = Log (show x ++ "++" ++ show y ++ "; ") (x++y)

foldr' logApp (logInit []) ["cat","dog","gnu"]
foldl' logApp (logInit []) ["cat","dog","gnu"]

Log "initial value = \"\"; \"gnu\"++\"\"; \"dog\"++\"gnu\"; \"cat\"++\"doggnu\"; " "catdoggnu"

Log "initial value = \"\"; \"\"++\"cat\"; \"cat\"++\"dog\"; \"catdog\"++\"gnu\"; " "catdoggnu"

**3.** Another classic example is using `foldl` to reverse a list. This uses what is essentially `(:)` as the accumulator function (we have to reverse the order of the arguments), and it shows up a difference in behaviour between `foldl` and `foldr`:

In [22]:
:t foldr
foldr (:) [] [1..4]
:t foldl
foldl (\xs x -> x:xs) [] [1..4]

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

[1,2,3,4]

foldl :: forall (t :: * -> *) b a. Foldable t => (b -> a -> b) -> b -> t a -> b

[4,3,2,1]

Use the Log monad together with `foldl'` and `foldr'` to log these functions. 

In [38]:
logCons x y = Log (show x ++ ":" ++ show y ++ "; ") (x:y)
foldr' logCons (logInit []) [1,2]
foldl' (\xs x -> logCons x xs) (logInit []) [1,2]

Line 3: Avoid lambda
Found:
\ xs x -> logCons x xs
Why not:
flip logCons

Log "initial value = []; 2:[]; 1:[2]; " [1,2]

Log "initial value = []; 1:[]; 2:[1]; " [2,1]

Explain why in this instance `foldl` reverses the list, while `foldr` leaves it unchanged. Use the list `[1,2]` as an example. 

Explanation here: `foldr` starts at the right (the last element of the list) and moves through successively cons'ing the elements it meets onto the front of the accumulator. It therefore reproduces the original list. `foldl` starts at the head of the list and moves down the list successively cons'ing the elements it meets onto the front of the accumulator. It therefore puts the head of the list at what will become the end of the result, and so on. Consequently it reverses the order of elements in the list. 

**4.** We move on to another monad, the use of State. Here is a StateTransformer monad.  

In [23]:
data StateTransformer s t = StateTransformer (s -> (s,t))

Line 1: Use newtype instead of data
Found:
data StateTransformer s t = StateTransformer (s -> (s, t))
Why not:
newtype StateTransformer s t = StateTransformer (s -> (s, t))

In [42]:
runST (StateTransformer f) = f

showRunST stf init = let (final,_) = runST stf init in "initial state is "++show init++"; final state is "++show final

instance Monad (StateTransformer s) where
  return t = StateTransformer (\s -> (s,t))
  (>>=) st1 g = StateTransformer (\s -> let (s1,t1) = runST st1 s in runST (g t1) s1)

instance Applicative (StateTransformer s) where
  pure = return
  (<*>) = ap
  
instance Functor (StateTransformer s) where
  fmap = liftM

Line 6: Use tuple-section
Found:
\ s -> (s, t)
Why not:
(, t)Line 6: Use tuple-section
Found:
\ s -> (s, t)
Why not:
(, t)

The first exercise is to use this monad. Consider the case when the state s is `Int`. Implement functions StateTransformer actions of type Int

In [40]:
stAdd :: Int -> StateTransformer Int ()
stAdd n = StateTransformer (\s -> (s+n,()))

stMult :: Int -> StateTransformer Int ()
stMult n = StateTransformer (\s -> (s*n,()))

Test them (you should get `"initial state is 0; final state is 15"`: 

In [43]:
com = do
  stAdd 3
  stMult 5

showRunST com 0
 

"initial state is 0; final state is 15"

Now combine the Log and the State Transformer. Complete the bind method: 

In [51]:
data LoggedStateTransformer s t = LoggedStateTransformer (String, s -> (s,t))

Line 1: Use newtype instead of data
Found:
data LoggedStateTransformer s t = LoggedStateTransformer (String,
                                                          s -> (s, t))
Why not:
newtype LoggedStateTransformer s t = LoggedStateTransformer (String,
                                                             s -> (s, t))

In [56]:
runLST (LoggedStateTransformer (s,f)) = f

instance Monad (LoggedStateTransformer s) where
  return t = LoggedStateTransformer ("",\s -> (s,t))
  (>>=) lst1 g = undefined
 
instance Applicative (LoggedStateTransformer s) where
  pure = return
  (<*>) = ap
  
instance Functor (LoggedStateTransformer s) where
  fmap = liftM

Line 4: Use tuple-section
Found:
\ s -> (s, t)
Why not:
(, t)Line 4: Use tuple-section
Found:
\ s -> (s, t)
Why not:
(, t)

Test this by adapting the previous example. 